In [2]:
#importing libraries
import pandas as pd
import numpy as np

#reading csv files
train = pd.read_csv('Data/Train.csv')
valid = pd.read_csv('Data/Valid.csv')             

#train_test split
x_tr, y_tr = train['text'].values, train['label'].values
x_val, y_val = valid['text'].values, valid['label'].values

In [6]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=100)
x_val_seq = pad_sequences(x_val_seq, maxlen=100)

In [7]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

112204


In [8]:
#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=100,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          33661200  
                                                                 
 lstm (LSTM)                 (None, 100, 128)          219648    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33,889,169
Trainable params: 33,889,169
Non-trainable params: 0
____________________________________________

In [9]:
import numpy as np

In [10]:
history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(y_val)),verbose=1,callbacks=[es,mc])

Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 0.4082 - acc: 0.8036
Epoch 1: val_acc improved from -inf to 0.86720, saving model to best_model.h5
313/313 [==============================] - 446s 1s/step - loss: 0.4082 - acc: 0.8036 - val_loss: 0.3102 - val_acc: 0.8672
Epoch 2/10
313/313 [==============================] - ETA: 0s - loss: 0.2051 - acc: 0.9210
Epoch 2: val_acc improved from 0.86720 to 0.86740, saving model to best_model.h5
313/313 [==============================] - 435s 1s/step - loss: 0.2051 - acc: 0.9210 - val_loss: 0.3211 - val_acc: 0.8674
Epoch 3/10
313/313 [==============================] - ETA: 0s - loss: 0.1042 - acc: 0.9628
Epoch 3: val_acc did not improve from 0.86740
313/313 [==============================] - 437s 1s/step - loss: 0.1042 - acc: 0.9628 - val_loss: 0.3978 - val_acc: 0.8634
Epoch 4/10
313/313 [==============================] - ETA: 0s - loss: 0.0506 - acc: 0.9828
Epoch 4: val_acc did not improve from 0.86740
313/313 [==========

In [11]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_val, batch_size=128)
print(val_acc)

40/40 [==============================] - 8s 188ms/step - loss: 0.3211 - acc: 0.8674
0.8673999905586243


In [12]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('D:/glove.6B.300d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [13]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix],input_length=100,trainable=False)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  


#Print summary of model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 300)          33661200  
                                                                 
 lstm_1 (LSTM)               (None, 100, 128)          219648    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33,889,169
Trainable params: 227,969
Non-trainable params: 33,661,200
____________________________________

In [15]:
history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(y_val)),verbose=1,callbacks=[es,mc])

Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 0.4475 - acc: 0.7813
Epoch 1: val_acc improved from -inf to 0.84880, saving model to best_model.h5
313/313 [==============================] - 196s 613ms/step - loss: 0.4475 - acc: 0.7813 - val_loss: 0.3506 - val_acc: 0.8488
Epoch 2/10
313/313 [==============================] - ETA: 0s - loss: 0.3396 - acc: 0.8499
Epoch 2: val_acc improved from 0.84880 to 0.86260, saving model to best_model.h5
313/313 [==============================] - 192s 615ms/step - loss: 0.3396 - acc: 0.8499 - val_loss: 0.3173 - val_acc: 0.8626
Epoch 3/10
313/313 [==============================] - ETA: 0s - loss: 0.3034 - acc: 0.8678
Epoch 3: val_acc improved from 0.86260 to 0.87040, saving model to best_model.h5
313/313 [==============================] - 193s 618ms/step - loss: 0.3034 - acc: 0.8678 - val_loss: 0.3019 - val_acc: 0.8704
Epoch 4/10
313/313 [==============================] - ETA: 0s - loss: 0.2694 - acc: 0.8848
Epoch 4: val_acc did 

In [16]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_val, batch_size=128)
print(val_acc)

40/40 [==============================] - 11s 240ms/step - loss: 0.2983 - acc: 0.8802
0.8802000284194946
